We'll use this script to compare lines from the two sites.

A reminder that for a double to be a greater value on PrizePicks than DraftKings, 
the two bets must multiply to a value lesser than three.

In [3]:
from draft_kings import get_draftkings_cbb_data# this will run the script that scrapes the data into the json file we will use
import sys
sys.path.append('..')  # Add the parent directory to the sys.path
from constants import *
from pp_scrape import get_data

In [4]:
import json, csv, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
COLLEGE_BASKETBALL = "20"
MLB = "2"
NBA = "3"

# get_draftkings_cbb_data() # this will run the script that scrapes the data into the json file we will use
get_data(NBA) # this will run the script that scrapes the data into the json file we will use

dict_keys(['adjusted_odds', 'board_time', 'custom_image', 'description', 'end_time', 'flash_sale_line_score', 'game_id', 'hr_20', 'is_promo', 'line_score', 'odds_type', 'projection_type', 'rank', 'refundable', 'start_time', 'stat_type', 'status', 'today', 'tv_channel', 'updated_at'])
{'adjusted_odds': None, 'board_time': '2024-07-09T03:51:00-04:00', 'custom_image': None, 'description': 'CHC', 'end_time': None, 'flash_sale_line_score': None, 'game_id': '3cd1efed-2db4-4a93-8edd-2e2757ea5770', 'hr_20': True, 'is_promo': False, 'line_score': 5, 'odds_type': 'standard', 'projection_type': 'Single Stat', 'rank': 1, 'refundable': True, 'start_time': '2024-07-09T18:35:00-04:00', 'stat_type': 'Pitcher Strikeouts', 'status': 'pre_game', 'today': True, 'tv_channel': None, 'updated_at': '2024-07-09T08:08:27-04:00'}
dict_keys(['adjusted_odds', 'board_time', 'custom_image', 'description', 'end_time', 'flash_sale_line_score', 'game_id', 'hr_20', 'is_promo', 'line_score', 'odds_type', 'projection_type

[{'name': 'Dean Kremer',
  'opponent': 'CHC',
  'Pitcher Strikeouts': 5,
  'Pitching Outs': 15.5,
  'Pitcher Fantasy Score': 27.5},
 {'name': 'Jameson Taillon',
  'opponent': 'BAL',
  'Pitcher Strikeouts': 4,
  'Pitching Outs': 17.5},
 {'name': 'Gunnar Henderson',
  'opponent': 'CHC',
  'Total Bases': 1.5,
  'Hitter Fantasy Score': 8.5},
 {'name': 'Christopher Morel', 'opponent': 'BAL', 'Hits+Runs+RBIs': 1.5},
 {'name': 'Jordan Westburg', 'opponent': 'CHC', 'Total Bases': 1.5},
 {'name': 'Dansby Swanson', 'opponent': 'BAL', 'Hits+Runs+RBIs': 1.5},
 {'name': 'Ian Happ', 'opponent': 'BAL', 'Hits+Runs+RBIs': 1.5},
 {'name': 'Heston Kjerstad', 'opponent': 'CHC', 'Hits+Runs+RBIs': 1.5},
 {'name': 'Nico Hoerner',
  'opponent': 'BAL',
  'Runs': 0.5,
  'Hitter Strikeouts': 0.5},
 {'name': 'Ben Lively', 'opponent': 'DET', 'Pitcher Strikeouts': 4},
 {'name': 'Kenta Maeda',
  'opponent': 'CLE',
  'Pitcher Strikeouts': 3,
  'Walks Allowed': 1.5,
  'Earned Runs Allowed': 2.5,
  'Hits Allowed': 5.5}

In [6]:
dk_lines = pd.read_csv('draftkings_mlb.csv', encoding='latin-1')
print(dk_lines['category'].value_counts())
# # list the items that appear multiple times
dk_lines['category'].value_counts()[dk_lines['category'].value_counts() > 1]
game_props = ["Game Lines", "Team Props", "Halves", "Game Props", "Total", "Innings", "Total Runs - 1st 5 Innings"]

player_lines = dk_lines[~dk_lines['category'].isin(game_props)]
player_lines = player_lines.dropna()
player_lines

category
Hits                               509
Strikeouts                         482
Runs Scored                        464
Doubles                            464
RBIs                               464
Total Bases                        463
Hits + Runs + RBIs                 462
Singles                            460
Batter Props                       449
Walks                              436
Stolen Bases                       278
Inning Run Line                    208
1st Run                            160
1st RBI                            160
1st Hit                            160
Team Total Runs - 1st X Innings    156
Earned Runs Allowed                 52
Game Props                          52
Pitcher Props                       52
Walks Allowed                       52
Hits Allowed                        50
Batters to the Plate                48
Pitches Thrown                      47
Outs Recorded                       46
Total                               30
Game Lines      

,providerOutcomeId,providerId,providerOfferId,label,oddsAmerican,oddsDecimal,oddsDecimalDisplay,oddsFractional,line,participant,participantType,main,sortOrder,tags,participants,category,prop_label
1022,0QA189537207#1756598755_13L84240Q1191372959Q20,2,189537207,Over,+1000,11,11.00,10/1,0.5,Aaron Hicks,Unknown,True,500,['MainPointLine'],"[{'id': '32486', 'name': 'Aaron Hicks', 'type'...",Batter Props,Aaron Hicks Home Runs
1023,0QA189537207#1756598756_13L84240Q1-672032808Q20,2,189537207,Under,-5000,1.02,1.02,1/50,0.5,Aaron Hicks,Unknown,True,502,['MainPointLine'],"[{'id': '32486', 'name': 'Aaron Hicks', 'type'...",Batter Props,Aaron Hicks Home Runs
1024,0QA189512953#1756380352_13L84240Q1191372959Q20,2,189512953,Over,+750,8.5,8.50,15/2,0.5,Adley Rutschman,Unknown,True,500,"['MainPointLine', '1stFavorite']","[{'id': '459732', 'name': 'Adley Rutschman', '...",Batter Props,Adley Rutschman Home Runs
1025,0QA189512953#1756380353_13L84240Q1-672032808Q20,2,189512953,Under,-2000,1.05,1.05,1/20,0.5,Adley Rutschman,Unknown,True,502,"['MainPointLine', '1stFavorite']","[{'id': '459732', 'name': 'Adley Rutschman', '...",Batter Props,Adley Rutschman Home Runs
1026,0QA189254093#1754994622_13L84240Q1191372959Q20,2,189254093,Over,+800,9,9.00,8/1,0.5,Anthony Rendon,Unknown,True,500,['MainPointLine'],"[{'id': '32800', 'name': 'Anthony Rendon', 'ty...",Batter Props,Anthony Rendon Home Runs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13928,0QA189240498#1754954006_13L84240Q1768026118Q20,2,189240498,Alex Wood,-130,1.76923077,1.76,10/13,Alex Wood,Player,32434,Alex Wood,1754957006,['1stFavorite'],"[{'id': '32434', 'name': 'Alex Wood', 'type': ...",H2H Props,1st Strikeout Thrown - Alex Wood v Shane Bieber
13929,0QA189243107#1754961901_13L84240Q11762371529Q20,2,189243107,Kyle Freeland,+475,5.75,5.75,19/4,Kyle Freeland,Player,32581,Kyle Freeland,1754962901,['1stFavorite'],"[{'id': '32581', 'name': 'Kyle Freeland', 'typ...",H2H Props,1st Strikeout Thrown - Zac Gallen v Kyle Freeland
13930,0QA189243107#1754961900_13L84240Q11876820397Q20,2,189243107,Zac Gallen,-900,1.11111112,1.11,1/9,Zac Gallen,Player,32144,Zac Gallen,1754964900,['1stFavorite'],"[{'id': '32144', 'name': 'Zac Gallen', 'type':...",H2H Props,1st Strikeout Thrown - Zac Gallen v Kyle Freeland
13931,0QA189234882#1754931856_13L84240Q1-685823980Q20,2,189234882,Brayan Bello,+475,5.75,5.75,19/4,Brayan Bello,Player,459744,Brayan Bello,1754932856,['1stFavorite'],"[{'id': '459744', 'name': 'Brayan Bello', 'typ...",H2H Props,1st Strikeout Thrown - Luis Castillo v Brayan ...


In [7]:
def get_categories(dk_db):
    ans = []
    combos = []
    for index, row in dk_db.iterrows():
        if row['participant'] in row['prop_label']:
            stat_tracked = row['prop_label'].replace(row['participant'], '').strip()
            if stat_tracked not in ans:
                if "+" in stat_tracked or "-" in stat_tracked:
                    combos.append(stat_tracked)
                ans.append(stat_tracked)
    return ans, combos



In [8]:
categories, combos = get_categories(player_lines)
player_lines['stat_tracked'] = player_lines.apply(lambda player: next((category.lower() for category in combos if category in player['prop_label']), None), axis=1)
player_lines['stat_tracked'] = player_lines.apply(lambda player: next((category.lower() for category in categories if category in player['prop_label'] and "+" not in player['prop_label']), player['stat_tracked']), axis=1)

player_lines = player_lines.dropna()
print(categories)
print(combos)



player_lines = player_lines[~player_lines['stat_tracked'].str.contains(':')]
player_lines = player_lines[~player_lines['stat_tracked'].str.contains('1st')]
player_lines = player_lines[player_lines['label'].isin(['Over', 'Under'])]

print(player_lines['stat_tracked'].value_counts())
print(player_lines['label'].value_counts())
print(player_lines['participant'].value_counts())

['Home Runs', 'Strikeouts Thrown', 'Hits - 1st Inning', 'Total Strikeouts Thrown - 1st Inning', 'Pitches Thrown - 1st Inning', 'Batters to the Plate - 1st Inning', ': Team Total Runs - 1st 3 Innings', ': Team Total Runs - 1st 5 Innings', ': Team Total Runs - 1st 7 Innings', 'Hits', 'Total Bases', 'RBIs', 'Runs', 'Hits + Runs + RBIs', 'Stolen Bases', 'Strikeouts', 'Singles', 'Doubles', 'Triples', 'Walks', 'Outs', 'Hits Allowed', 'Earned Runs Allowed', 'Walks Allowed']
['Hits - 1st Inning', 'Total Strikeouts Thrown - 1st Inning', 'Pitches Thrown - 1st Inning', 'Batters to the Plate - 1st Inning', ': Team Total Runs - 1st 3 Innings', ': Team Total Runs - 1st 5 Innings', ': Team Total Runs - 1st 7 Innings', 'Hits + Runs + RBIs']
stat_tracked
runs                  516
hits                  512
walks                 488
rbis                  464
doubles               464
total bases           463
hits + runs + rbis    462
strikeouts            462
singles               460
home runs         

In [9]:
final_dk_db = {} #dic so we can combine lines for each player

for index, row in player_lines.iterrows():
    if row['participant'] not in final_dk_db:
        final_dk_db[row['participant']] = {
            'name': row['participant']
        }
    final_dk_db[row['participant']][row['stat_tracked']] = row['line']
    final_dk_db[row['participant']][row['stat_tracked'] + '_' + row['label']] = row['oddsDecimal']

final_dk_db = pd.DataFrame(final_dk_db).T



In [10]:
final_dk_db

,name,home runs,home runs_Over,home runs_Under,hits,hits_Over,hits_Under,total bases,total bases_Over,total bases_Under,...,walks_Under,triples,triples_Over,triples_Under,strikeouts thrown,strikeouts thrown_Over,strikeouts thrown_Under,outs,outs_Over,outs_Under
Aaron Hicks,Aaron Hicks,0.5,11,1.02,0.5,1.76923077,1.95238096,0.5,1.76923077,1.95238096,...,1.76923077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adley Rutschman,Adley Rutschman,0.5,8.5,1.05,0.5,1.43478261,2.7,1.5,2.4,1.52631579,...,2.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Anthony Rendon,Anthony Rendon,0.5,9,1.05,0.5,1.44444445,2.65,1.5,2.45,1.5,...,1.57142858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Anthony Santander,Anthony Santander,0.5,5.5,1.125,0.5,1.51282052,2.45,1.5,2.45,1.52631579,...,1.76923077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Austin Hays,Austin Hays,0.5,9,1.04,0.5,1.46511628,2.6,1.5,2.55,1.47619048,...,1.45454546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Esteury Ruiz,Esteury Ruiz,NaN,NaN,NaN,0.5,1.8,1.95238096,0.5,1.8,1.95238096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nick Allen,Nick Allen,NaN,NaN,NaN,0.5,1.74074075,2.05,0.5,1.74074075,2.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Steven Kwan,Steven Kwan,NaN,NaN,NaN,1.5,2.7,1.42553192,1.5,2.25,1.60606061,...,1.41666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Will Brennan,Will Brennan,NaN,NaN,NaN,0.5,1.46511628,2.6,0.5,1.46511628,2.6,...,1.17391305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
prizepicks = pd.read_csv("pp_lines_cbb.csv")

merged = pd.merge(prizepicks, final_dk_db, on='name')
#get a list of all the column names
cols = merged.columns.tolist()
pp_to_dk_stats.keys()


dict_keys(['Rebounds', 'Points', 'Pts+Asts', 'Pts+Rebs', 'Pts+Rebs+Asts', 'Assists', 'Rebs+Asts', 'Fantasy Score', '3-PT Made', 'Hits Allowed', 'Pitcher Strikeouts', 'Pitching Outs', 'Walks Allowed', 'Hits+Runs+RBIS', 'Hitter Strikeouts', 'Total Bases', 'Runs', 'Walks'])

In [12]:
results = []

for index, row in merged.iterrows():
    # Access the values of each column in the current row
    name = row['name']

    for pp_stat_type in pp_to_dk_stats.keys():
        # failsafe 
        if pp_stat_type not in row:
            continue

        if float(row[pp_stat_type]) <= 0:
            continue

        if pp_to_dk_stats[pp_stat_type] not in row:
            continue
    

        if float(row[pp_to_dk_stats[pp_stat_type]]) <= 0:
            continue

        # Create a dictionary to store the result
        result_dict = {
            'name': name,
            'stat': pp_stat_type,
            'line': row[pp_stat_type],
            'dk_line': float(row[pp_to_dk_stats[pp_stat_type]]),
            'dk_over_line': row[pp_to_dk_stats[pp_stat_type] + '_Over'],
            'dk_under_line': row[pp_to_dk_stats[pp_stat_type] + '_Under']
        }

        # Get the dk stat type that corresponds to the current PP stat type
        dk_stat_type = pp_to_dk_stats[pp_stat_type]

        # Get the value of the FBref stat type for the current player
        dk_stat_value = row[dk_stat_type]



        # Append the dictionary to the results list
        results.append(result_dict)

results_df = pd.DataFrame(results)
print(results_df.head())


results_df = results_df.dropna()
# results_df = results_df.dropna() 
# # get data types for each column 
# print(results_df.dtypes)



           name                stat  line  dk_line dk_over_line dk_under_line
0  Jose Ramirez        Hits Allowed   NaN      0.5    1.3773585           2.9
1  Jose Ramirez  Pitcher Strikeouts   NaN      NaN          NaN           NaN
2  Jose Ramirez       Pitching Outs   NaN      NaN          NaN           NaN
3  Jose Ramirez       Walks Allowed   NaN      0.5          2.5    1.48780488
4  Jose Ramirez         Total Bases   1.5      1.5         2.05    1.71428572


In [13]:
results_df

,name,stat,line,dk_line,dk_over_line,dk_under_line
4,Jose Ramirez,Total Bases,1.5,1.5,2.05,1.71428572
5,Jose Ramirez,Runs,0.5,0.5,2.05,1.74074075
19,Andres Gimenez,Runs,0.5,0.5,2.45,1.5
25,Fernando Tatis Jr.,Total Bases,1.5,1.5,2.1,1.68965518
26,Fernando Tatis Jr.,Runs,0.5,0.5,1.95238096,1.8
102,Rafael Devers,Total Bases,1.5,1.5,1.95238096,1.76923077
109,Adley Rutschman,Total Bases,1.5,1.5,2.4,1.52631579
116,Jarren Duran,Total Bases,1.5,1.5,2.1,1.66666667
123,Anthony Santander,Total Bases,1.5,1.5,2.45,1.52631579
130,Tyler O'Neill,Total Bases,1.5,0.5,1.6451613,2.15


In [14]:
results_df = results_df[results_df['line'] == results_df['dk_line']]
results_df.sort_values(by='dk_over_line', ascending=True)

,name,stat,line,dk_line,dk_over_line,dk_under_line
249,Tyler Wade,Total Bases,0.5,0.5,1.6451613,2.15
319,Nick Fortes,Total Bases,0.5,0.5,1.66666667,2.1
306,Jonathan India,Runs,0.5,0.5,1.8,1.95238096
242,Mike Tauchman,Total Bases,0.5,0.5,1.8,1.95238096
159,Jose Altuve,Runs,0.5,0.5,1.83333334,1.90909091
26,Fernando Tatis Jr.,Runs,0.5,0.5,1.95238096,1.8
102,Rafael Devers,Total Bases,1.5,1.5,1.95238096,1.76923077
4,Jose Ramirez,Total Bases,1.5,1.5,2.05,1.71428572
5,Jose Ramirez,Runs,0.5,0.5,2.05,1.74074075
272,Ian Happ,Walks,0.5,0.5,2.05,1.71428572


In [15]:
re

NameError: name 're' is not defined